# Assignment 2 Part 5

Abhinav Kumar
10/13/2025

Similarity Search - Querying PGVector for the top-10 nearest neighbors of a given embedding

In [ ]:

import psycopg2, pandas as pd
from psycopg2.extras import RealDictCursor

PG = dict(host="localhost", port=5432, dbname="mvtec", user="postgres", password="postgres", table="mvtec_embeddings")

def get_conn():
    return psycopg2.connect(host=PG["host"], port=PG["port"], dbname=PG["dbname"], user=PG["user"], password=PG["password"])

def topk(vec, k=10):
    sql = f"""
    SELECT category, split, image_path, label, embedding <-> %s::vector AS dist
    FROM {PG['table']}
    ORDER BY embedding <-> %s::vector
    LIMIT {k};
    """
    v = list(map(float, vec))
    with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(sql, (v, v))
        rows = cur.fetchall()
    return pd.DataFrame(rows)


In [ ]:

with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(f"SELECT embedding FROM {PG['table']} LIMIT 1;")
    one = cur.fetchone()
query_vec = one["embedding"]
df = topk(query_vec, k=10)
df
